请写一个整数计算器，支持加减乘三种运算和括号。

#### 示例1
```
输入 "1+2"
返回值 3
```

#### 示例2
```
输入 "(2*(3-4))*5"
返回值 -10
```

#### 示例3
```
输入 "3+2*3*4-1"
返回值 26
```

参考[leetcode 150](https://leetcode-cn.com/problems/evaluate-reverse-polish-notation/)和[leetcode 224](https://leetcode-cn.com/problems/basic-calculator/)

#### 中缀表达式转后缀表达式（逆波兰表达式）

1. 初始化两个栈：运算符栈`stack`和储存中间结果的栈`ast`；
2. 从左至右扫描中缀表达式；
3. 遇到操作数时，将其压`ast`；
4. 遇到运算符时，比较其与s1栈顶运算符的优先级：
    1. 如果`stack`为空，或栈顶运算符为左括号“(”，则直接将此运算符入栈；
    2. 否则，若优先级比栈顶运算符的高，也将运算符压入`stack`；
    3. 否则，将`stack`栈顶的运算符弹出并压入到`ast`中，再次转到(4.1)与`stack`中新的栈顶运算符相比较； 
5. 遇到括号时：　
    1. 如果是左括号"("，则直接压入`stack`　
    2. 如果是右括号")"，则依次弹出`stack`栈顶的运算符，并压入`ast`，直到遇到左括号为止，此时将这一对括号丢弃6.重复步骤2至5，直到表达式的最右边
7. 将`stack`中剩余的运算符依次弹出并压入`ast`
8. 依次弹出`ast`中的元素并输出，结果的逆序即为中缀表达式对应的后缀表达式

In [50]:
def tokenize(s):
    tokens, digitStart, sign = [], -1, 1
    signleads = {'+', "*", "("}
    for i, c in enumerate(s):
        if c.isdigit():
            if digitStart <= -1:
                digitStart = i
        else:
            if digitStart >= 0:
                if sign == -1:
                    tokens.append('-' + s[digitStart:i])
                else:
                    tokens.append(s[digitStart:i])
                digitStart = -1
            if c == '+' or c == '*' or c == '(' or c == ')':
                tokens.append(c)
                sign = 1
            elif c == '-':
                if len(tokens) == 0 or tokens[-1] in signleads:
                    sign = -1
                else:
                    tokens.append(c)
            elif c == ' ':
                continue
            else:
                print(c)
        
    if digitStart > -1:
        tokens.append(s[digitStart:])
                
    return tokens

print(tokenize("-2+ 1"))
print(tokenize("1 + 1"))
print(tokenize("2-1 + 2"))
print(tokenize("(1+(4+5+2)-3)+(6+8)"))
print(tokenize("216-18 + -3 + (2-1)+ 100-50"))

['-2', '+', '1']
['1', '+', '1']
['2', '-', '1', '+', '2']
['(', '1', '+', '(', '4', '+', '5', '+', '2', ')', '-', '3', ')', '+', '(', '6', '+', '8', ')']
['216', '-', '18', '+', '-3', '+', '(', '2', '-', '1', ')', '+', '100', '-', '50']


In [62]:
def parse(tokens):
    ast, stack = [], []
    for i, t in enumerate(tokens):
        if t.isdigit() or t[0] == '-' and t[1:].isdigit():
            ast.append(t)
        elif t == '+' or t == '-' or t == '*':
            if len(stack) == 0 or stack[-1] == '(':
                stack.append(t)
            else:
                top = stack[-1]
                if (top == '+' or top == '-') and t == '*':
                    stack.append(t)
                else:
                    ast.append(stack.pop())
                    stack.append(t)
        elif t == '(':
            stack.append(t)
        elif t == ')':
            while len(stack) > 0:
                top = stack.pop()
                if top == '(':
                    break
                else:
                    ast.append(top)
    
    while len(stack) > 0:
        ast.append(stack.pop())
        
    return ast

In [63]:
import operator

operators = {'+': operator.add, '-': operator.sub, '*': operator.mul}

def evaluate(ast):
    op = ast.pop()
    if op in operators:
        right = evaluate(ast)
        left = evaluate(ast)
        return operators[op](left, right)
    else:
        return int(op)

In [64]:
def solve(s):
    tokens = tokenize(s)
    print(tokens)
    ast = parse(tokens)
    print(ast)
    return evaluate(ast)

In [65]:
solve("-2 + 1")

['-2', '+', '1']
['-2', '1', '+']


-1

In [66]:
solve("1+2")

['1', '+', '2']
['1', '2', '+']


3

In [67]:
solve("(2*(3-4))*5")

['(', '2', '*', '(', '3', '-', '4', ')', ')', '*', '5']
['2', '3', '4', '-', '*', '5', '*']


-10

In [68]:
solve("3+2*3*4-1")

['3', '+', '2', '*', '3', '*', '4', '-', '1']
['3', '2', '3', '*', '4', '*', '1', '-', '+']


26

In [72]:
solve("9+(3-1)*3+10 * 2")

['9', '+', '(', '3', '-', '1', ')', '*', '3', '+', '10', '*', '2']
['9', '3', '1', '-', '3', '*', '10', '2', '*', '+', '+']


35

In [71]:
solve("((10 * (6 * ((9 + 3) * -11))) + 17) + 5") # 10 6 9 3 + -11 * * * 17 + 5 +

['(', '(', '10', '*', '(', '6', '*', '(', '(', '9', '+', '3', ')', '*', '-11', ')', ')', ')', '+', '17', ')', '+', '5']
['10', '6', '9', '3', '+', '-11', '*', '*', '*', '17', '+', '5', '+']


-7898